In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
statisticCategory = input("Which statistical category (rushing/passing):")
season = input("For which year: ")

Which statistical category (rushing/passing):rushing
For which year: 2011


In [3]:
# Datset could have multiple pages.  We will set on page 1 to begin
p = 1

In [4]:
payload = {"statisticCategory":statisticCategory.upper(),"seasonType":"REG","d-447263-p":str(p),"season":season}
url = 'http://www.nfl.com/stats/categorystats'
response = requests.get(url,params=payload)


In [5]:
print("Response:", response.status_code,response.url) #200 means it went through


Response: 200 http://www.nfl.com/stats/categorystats?statisticCategory=RUSHING&seasonType=REG&d-447263-p=1&season=2011


In [6]:
soup = BeautifulSoup(response.text,'html.parser')

In [7]:
#The website table is partitioned into multiple pages.  We need to know how many pages to iterate

pagingText = soup.find("span", {"class": "linkNavigation floatRight"})

pagingLinks = pagingText.findAll("a")
pages = len(pagingLinks)
print("Number of page links:",len(pagingLinks))

Number of page links: 7


In [8]:
#Get Header for the table
columnHeader = []
header = soup.find('table', attrs={'id':'result'}).find_all('th')
for c in header:
    columnHeader.append(c.text)
columnHeader = [c.strip('\n') for c in columnHeader]


In [9]:
tableRows = []

for pg in range(1,pages+1):
    
    #pause our code for a second so that we are not spamming the website with requests. This helps us avoid getting flagged as a spammer.
    time.sleep(1)
    
    #update page number in payload
    payload["d-447263-p"] = pg
    response = requests.get(url,params=payload)
    print("Page:",pg,"url:",response.url)
    soup = BeautifulSoup(response.text,'html.parser')
    table = soup.find('table', attrs={'id':'result'})
    rows = table.find_all('tr')

    for row in rows:
        cols = row.find_all('td')
        if cols:
            cols = [ele.text.strip() for ele in cols]
            tableRows.append([ele for ele in cols if ele]) # Get rid of empty values
    
   


Page: 1 url: http://www.nfl.com/stats/categorystats?statisticCategory=RUSHING&seasonType=REG&d-447263-p=1&season=2011
Page: 2 url: http://www.nfl.com/stats/categorystats?statisticCategory=RUSHING&seasonType=REG&d-447263-p=2&season=2011
Page: 3 url: http://www.nfl.com/stats/categorystats?statisticCategory=RUSHING&seasonType=REG&d-447263-p=3&season=2011
Page: 4 url: http://www.nfl.com/stats/categorystats?statisticCategory=RUSHING&seasonType=REG&d-447263-p=4&season=2011
Page: 5 url: http://www.nfl.com/stats/categorystats?statisticCategory=RUSHING&seasonType=REG&d-447263-p=5&season=2011
Page: 6 url: http://www.nfl.com/stats/categorystats?statisticCategory=RUSHING&seasonType=REG&d-447263-p=6&season=2011
Page: 7 url: http://www.nfl.com/stats/categorystats?statisticCategory=RUSHING&seasonType=REG&d-447263-p=7&season=2011


In [10]:
resultsDF = pd.DataFrame(tableRows, columns=columnHeader)
display(resultsDF.head(),resultsDF.shape)


,Rk,Player,Team,Pos,Att,Att/G,Yds,Avg,Yds/G,TD,Lng,1st,1st%,20+,40+,FUM
0,1,Maurice Jones-Drew,JAC,RB,343,21.4,"1,606",4.7,100.4,8,56,75,21.9,9,3,3
1,2,Arian Foster,HOU,RB,278,21.4,"1,224",4.4,94.2,10,43,58,20.9,7,2,4
2,3,Fred Jackson,BUF,RB,170,17.0,934,5.5,93.4,6,80T,40,23.5,10,3,2
3,4,Darren McFadden,OAK,RB,113,16.1,614,5.4,87.7,4,70T,23,20.4,8,3,1
4,5,LeSean McCoy,PHI,RB,273,18.2,"1,309",4.8,87.3,17,60,84,30.8,14,2,1


(319, 16)

In [11]:
#Save results to Excel
baseFileName = "NFL_Stats_Demo_"

resultsDF.to_excel(baseFileName + statisticCategory + "_" + season + ".xlsx",index=False)